In [1]:
# bob.refresh {"cron": "10,35 8,11,19,21 * * *", "timeout": 40000}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
hc = bob.HealthCheck()
hc_key = "99c0c879-0fe1-45c5-b5a6-95f0c2b1b203"
if use_healthchecks:
    step_1 = hc.start(hc_key)
    print(step_1)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-30


In [4]:
#--- Covid-19 France data updated everyday at 19:00PM
%run "covid-19_france.ipynb"

Script execution completed at 30/04/2020 10:42:34. Time: --- 0.9102411270141602 secnds ---
Dataframe FRANCE_INIT successfully save in database covid-19-dev in MongoDB. Time: --- 0.37319207191467285 secnds ---
Script execution completed at 30/04/2020 10:42:34. Time: --- 0.011027097702026367 secnds ---
Dataframe FRANCE_DB_CONCAT successfully save in database covid-19-dev in MongoDB. Time: --- 0.4924743175506592 secnds ---
Script execution completed at 30/04/2020 10:42:34. Time: --- 0.07040262222290039 secnds ---
Dataframe FRANCE_DB_CONSO successfully save in database covid-19-dev in MongoDB. Time: --- 0.735497236251831 secnds ---
Script execution completed at 30/04/2020 10:42:34. Time: --- 0.2575082778930664 secnds ---
Dataframe FRANCE_DB_ALL successfully save in database covid-19-dev in MongoDB. Time: --- 13.197204113006592 secnds ---
Script execution completed at 30/04/2020 10:42:34. Time: --- 0.10339093208312988 secnds ---
Dataframe FRANCE_DB_TREND successfully save in database covid-

In [5]:
#--- Load data source
start_time = time.time()
db_all_fr = get_datasource('FRANCE_DB_ALL')
db_trend_fr = get_datasource('FRANCE_DB_TREND')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 30/04/2020 10:43:06. Time: --- 0.18065595626831055 secnds ---


In [6]:
#-- Covid-19 - DATA.GOUV.FR TENDANCES
start_time = time.time()
def domain_105(df,domain_num):
    #-- Create Datamodel
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'GROUPS': domain['STATUS_NAME'],
        'GROUPS_ORDER': domain['STATUS_ORDER'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'BOTTOM_FILTER': 'Depuis le début',
        'UPPER_FILTER_M': domain['METRIC'],
        'UPPER_FILTER_R': domain['LABEL'],
        'LABEL_GROUPS': domain['LABEL_GROUPS'],
        'VALUE': domain['VALUE'],
        'UNIT_VALUE': domain['UNIT'],
        'PRECISION': domain['PRECISION'],
    })
    
    #-- Rename upper filter
    domain.loc[domain['UPPER_FILTER_M'] == 'VALUE','UPPER_FILTER_M'] = 'EVOLUTION'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARV','UPPER_FILTER_M'] = 'VARIATION EN NB'
    domain.loc[domain['UPPER_FILTER_M'] == 'VARP','UPPER_FILTER_M'] = 'VARIATION EN %'
    
    #-- Create new date scenario in BOTTOM_FILTER
    df1 = get_lastdays(domain, 15,"15 derniers jours",'BOTTOM_FILTER')
    domain = pd.concat([domain,df1],axis=0)
    
    #-- Filter upperfilter value to get France and detailed by region
    domain_dep = domain[domain['LABEL_GROUPS'] != 'France']
    domain_fra = domain[domain['UPPER_FILTER_R'] == 'France']
    domain = pd.concat([domain_dep,domain_fra],axis=0)
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    
    #-- Create upper filter domain
    upper_filter = domain[['UPPER_FILTER_R']].drop_duplicates().sort_values(by='UPPER_FILTER_R', ascending=True).reset_index(drop=True)
    upper_filter['ORDER'] = upper_filter.index + 1
    upper_filter.loc[upper_filter['UPPER_FILTER_R'] == 'France', 'ORDER'] = 0
    upper_filter = upper_filter.sort_values(by='ORDER',ascending=True)
    upper_filter.to_csv(output_folder + '105_UPPER_FILTER.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(upper_filter,'105_UPPER_FILTER',db_app,True)
    return domain

domain105 = domain_105(db_trend_fr,'105')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain105

Dataframe 105 successfully save in database app-wsr-dev in MongoDB. Time: --- 15.213393449783325 secnds ---
Dataframe 105_UPPER_FILTER successfully save in database app-wsr-dev in MongoDB. Time: --- 0.01755499839782715 secnds ---
Script execution completed at 30/04/2020 10:43:22. Time: --- 16.067234754562378 secnds ---


,ENTITY,SCENARIO,GROUPS,GROUPS_ORDER,DATE_ORDER,BOTTOM_FILTER,UPPER_FILTER_M,UPPER_FILTER_R,LABEL_GROUPS,VALUE,UNIT_VALUE,PRECISION,LAST_UPDATE
0,France,29/04/2020,Hospitalisations,1.0,20200429,Depuis le début,EVOLUTION,31 - Haute-Garonne,Occitanie,131.000000,cas,",.0f",30/04/2020 10:43:06
1,France,29/04/2020,Hospitalisations,1.0,20200429,Depuis le début,EVOLUTION,32 - Gers,Occitanie,35.000000,cas,",.0f",30/04/2020 10:43:06
2,France,29/04/2020,Hospitalisations,1.0,20200429,Depuis le début,EVOLUTION,33 - Gironde,Nouvelle-Aquitaine,234.000000,cas,",.0f",30/04/2020 10:43:06
3,France,29/04/2020,Hospitalisations,1.0,20200429,Depuis le début,EVOLUTION,34 - Hérault,Occitanie,110.000000,cas,",.0f",30/04/2020 10:43:06
4,France,29/04/2020,Hospitalisations,1.0,20200429,Depuis le début,EVOLUTION,35 - Ille-et-Vilaine,Bretagne,150.000000,cas,",.0f",30/04/2020 10:43:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69763,France,15/04/2020,Retours à domicile,3.0,20200415,15 derniers jours,VARIATION EN NB,France,France,2148.000000,cas,"+,.0f",30/04/2020 10:43:06
69764,France,15/04/2020,Hospitalisations,1.0,20200415,15 derniers jours,VARIATION EN %,France,France,-1.581028,%,"+,.2f",30/04/2020 10:43:06
69765,France,15/04/2020,Décès à l’hôpital,4.0,20200415,15 derniers jours,VARIATION EN %,France,France,5.074538,%,"+,.2f",30/04/2020 10:43:06
69766,France,15/04/2020,En réanimation,2.0,20200415,15 derniers jours,VARIATION EN %,France,France,-4.061221,%,"+,.2f",30/04/2020 10:43:06


In [7]:
#-- Covid-19 - DATA.GOUV.FR CLASSEMENT
start_time = time.time()
def domain_107(df,domain_num):
    domain = df.copy()
    domain = pd.DataFrame({
        'ENTITY': 'France',
        'SCENARIO': domain['SCENARIO'],
        'DATE_ORDER': domain['DATE_ORDER'],
        'CHILD': domain['LABEL'],
        'PARENT': domain['LABEL_GROUPS'],
        'UPPER_FILTER_M': domain['STATUS_NAME'],
        'ORDER':domain['STATUS_ORDER'],
        'VALUE': domain['VALUE'],
        'VAR': domain['VARV'],
        'VARP': domain['VARP'],
        'UNIT_VARP': ' %',
        'PRECISION_VALUE': ',.0f',
        'PRECISION_VAR': '+,.0f',
        'PRECISION_VARP': '+,.2f',
    })
    
    domain.loc[domain['PARENT'] == 'France', 'PARENT'] = 'root'
    
    #-- Reset indexes & save domain
    domain = domain.reset_index(drop=True)
    domain['LAST_UPDATE']= datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    domain.to_csv(output_folder + domain_num + '.csv', sep=';')
    if use_mongo:
        bob.mongo.save_df(domain,domain_num,db_app,True)
    return domain

domain107 = domain_107(db_all_fr,'107')
print("Script execution completed at " + datetime.now().strftime('%d/%m/%Y %H:%M:%S') + ". Time: --- %s secnds ---" % (time.time() - start_time))
domain107

Dataframe 107 successfully save in database app-wsr-dev in MongoDB. Time: --- 13.157564878463745 secnds ---
Script execution completed at 30/04/2020 10:43:35. Time: --- 13.340311050415039 secnds ---


,ENTITY,SCENARIO,DATE_ORDER,CHILD,PARENT,UPPER_FILTER_M,ORDER,VALUE,VAR,VARP,UNIT_VARP,PRECISION_VALUE,PRECISION_VAR,PRECISION_VARP,LAST_UPDATE
0,France,29/04/2020,20200429,Île-de-France,root,En réanimation,2.0,1680,-121,-6.718490,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
1,France,29/04/2020,20200429,Île-de-France,root,Hospitalisations,1.0,11085,-264,-2.326196,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
2,France,29/04/2020,20200429,31 - Haute-Garonne,Occitanie,Hospitalisations,1.0,131,-10,-7.092199,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
3,France,29/04/2020,20200429,32 - Gers,Occitanie,Hospitalisations,1.0,35,-5,-12.500000,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
4,France,29/04/2020,20200429,33 - Gironde,Nouvelle-Aquitaine,Hospitalisations,1.0,234,-20,-7.874016,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20155,France,19/03/2020,20200319,87 - Haute-Vienne,Nouvelle-Aquitaine,Retours à domicile,3.0,2,0,0.000000,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
20156,France,19/03/2020,20200319,80 - Somme,Hauts-de-France,Retours à domicile,3.0,12,3,33.333333,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
20157,France,19/03/2020,20200319,79 - Deux-Sèvres,Nouvelle-Aquitaine,Retours à domicile,3.0,3,1,50.000000,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22
20158,France,19/03/2020,20200319,78 - Yvelines,Île-de-France,Retours à domicile,3.0,7,2,40.000000,%,",.0f","+,.0f","+,.2f",30/04/2020 10:43:22


In [8]:
if use_healthchecks:
    step_2 = hc.done(hc_key)
    print(step_2)

Done ==> send to https://health.cashstory.com/ping/99c0c879-0fe1-45c5-b5a6-95f0c2b1b203, 2020-04-30
